In [1]:
# Import Liabraries
import simfin as sf
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from collections import Counter

In [2]:
# Set API Key 
sf.set_api_key('ZLj424KDiohtSc0u9bVaF7mIeIemoAir')

In [3]:
# Make directory for data 
sf.set_data_dir('~/simfin_data/')

### Using simfin API to pull the earning information

In [57]:
# Loading financial ratio data in a DataFrame
ratios_df = sf.load_derived(variant='quarterly-asreported', market='us')
print(ratios_df.head())

Dataset "us-derived-quarterly-asreported" on disk (3 days old).
- Loading from disk ... Done!
                    SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2010-07-31      45846      USD         2010            Q3   2010-10-06   
       2010-10-31      45846      USD         2010            Q4   2010-12-20   
       2011-01-31      45846      USD         2011            Q1   2011-03-09   
       2011-04-30      45846      USD         2011            Q2   2011-06-07   
       2011-07-31      45846      USD         2011            Q3   2011-09-07   

                   Restated Date       EBITDA  Total Debt  Free Cash Flow  \
Ticker Report Date                                                          
A      2010-07-31     2011-09-07  175000000.0  3678000000     174000000.0   
       2010-10-31     2012-12-20  270000000.0  3691000000     258000000.0   
       2011-01-31     2012-03-

In [58]:
pd.DataFrame(ratios_df)

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2010-07-31      45846      USD         2010            Q3   2010-10-06   
       2010-10-31      45846      USD         2010            Q4   2010-12-20   
       2011-01-31      45846      USD         2011            Q1   2011-03-09   
       2011-04-30      45846      USD         2011            Q2   2011-06-07   
       2011-07-31      45846      USD         2011            Q3   2011-09-07   
...                      ...      ...          ...           ...          ...   
ZYXI   2020-09-30     171401      USD         2020            Q3   2020-10-27   
       2020-12-31     171401      USD         2020            Q4   2021-02-25   
       2021-03-31     171401      USD         2021            Q1   2021-04-29   
       2021-06-30     171401      USD         2021            Q2   2021-07-29   
       2021-09-30     171401      USD         2021            Q3   2021-11-02   

                   Restated Date       EBITDA  Total Debt  Free Cash Flow  \
Ticker Report Date                                                          
A      2010-07-31     2011-09-07  175000000.0  3678000000     174000000.0   
       2010-10-31     2012-12-20  270000000.0  3691000000     258000000.0   
       2011-01-31     2012-03-05  274000000.0  2139000000      65000000.0   
       2011-04-30     2012-06-04  330000000.0  2144000000     393000000.0   
       2011-07-31     2012-09-05  344000000.0  2168000000      74000000.0   
...                          ...          ...         ...             ...   
ZYXI   2020-09-30     2020-10-27    1914000.0           0             NaN   
       2020-12-31     2021-02-25    2827000.0           0      -3070000.0   
       2021-03-31     2021-04-29    -594000.0           0      -6150000.0   
       2021-06-30     2021-07-29    4422000.0           0       1714000.0   
       2021-09-30     2021-11-02    8641000.0           0       6443000.0   

                    Gross Profit Margin  ...  Sales Per Share  \
Ticker Report Date                       ...                    
A      2010-07-31               0.52384  ...          3.98847   
       2010-10-31               0.52919  ...          4.58140   
       2011-01-31               0.53720  ...          4.37752   
       2011-04-30               0.53667  ...          4.83285   
       2011-07-31               0.52750  ...          4.85920   
...                                 ...  ...              ...   
ZYXI   2020-09-30               0.78548  ...          0.52791   
       2020-12-31               0.77899  ...          0.66900   
       2021-03-31               0.75604  ...          0.62961   
       2021-06-30               0.76575  ...          0.81016   
       2021-09-30               0.80346  ...          0.90956   

                    Equity Per Share  Free Cash Flow Per Share  \
Ticker Report Date                                               
A      2010-07-31            8.07493                   0.50144   
       2010-10-31            9.38372                   0.75000   
       2011-01-31            9.60231                   0.18732   
       2011-04-30           11.39193                   1.13256   
       2011-07-31           12.04598                   0.21264   
...                              ...                       ...   
ZYXI   2020-09-30            1.43170                       NaN   
       2020-12-31            1.48550                  -0.08021   
       2021-03-31            1.46534                  -0.16049   
       2021-06-30            1.49644                   0.04476   
       2021-09-30            1.65730                   0.16847   

                    Dividends Per Share  Piotroski F-Score  \
Ticker Report Date                                           
A      2010-07-31                   NaN                NaN   
       2010-10-31                   0.0                6.0   
       2011

In [59]:
# Drop columns which will not add values
    # SimFinID has no relation to stock performance
    # All stock ratios are in US dollars
    # Restated date
    # Dividends Per Share and Dividends Payout Ratio filled with NaN or 0.0
ratios_df.drop(columns=['SimFinId', 'Currency', 'Restated Date', 'Dividends Per Share', 'Dividend Payout Ratio'], axis=1, inplace=True)
ratios_df

Fiscal Year Fiscal Period Publish Date       EBITDA  \
Ticker Report Date                                                        
A      2010-07-31          2010            Q3   2010-10-06  175000000.0   
       2010-10-31          2010            Q4   2010-12-20  270000000.0   
       2011-01-31          2011            Q1   2011-03-09  274000000.0   
       2011-04-30          2011            Q2   2011-06-07  330000000.0   
       2011-07-31          2011            Q3   2011-09-07  344000000.0   
...                         ...           ...          ...          ...   
ZYXI   2020-09-30          2020            Q3   2020-10-27    1914000.0   
       2020-12-31          2020            Q4   2021-02-25    2827000.0   
       2021-03-31          2021            Q1   2021-04-29    -594000.0   
       2021-06-30          2021            Q2   2021-07-29    4422000.0   
       2021-09-30          2021            Q3   2021-11-02    8641000.0   

                    Total Debt  Free Cash Flow  Gross Profit Margin  \
Ticker Report Date                                                    
A      2010-07-31   3678000000     174000000.0              0.52384   
       2010-10-31   3691000000     258000000.0              0.52919   
       2011-01-31   2139000000      65000000.0              0.53720   
       2011-04-30   2144000000     393000000.0              0.53667   
       2011-07-31   2168000000      74000000.0              0.52750   
...                        ...             ...                  ...   
ZYXI   2020-09-30            0             NaN              0.78548   
       2020-12-31            0      -3070000.0              0.77899   
       2021-03-31            0      -6150000.0              0.75604   
       2021-06-30            0       1714000.0              0.76575   
       2021-09-30            0       6443000.0              0.80346   

                    Operating Margin  Net Profit Margin  Return on Equity  \
Ticker Report Date                                                          
A      2010-07-31            0.08309            0.14812           0.07295   
       2010-10-31            0.12881            0.18528           0.09023   
       2011-01-31            0.13891            0.12706           0.05780   
       2011-04-30            0.15862            0.11926           0.05049   
       2011-07-31            0.16617            0.19515           0.07857   
...                              ...                ...               ...   
ZYXI   2020-09-30            0.07036            0.06656           0.02458   
       2020-12-31            0.08971            0.06979           0.03143   
       2021-03-31           -0.04480           -0.02926          -0.01257   
       2021-06-30            0.12298            0.09052           0.04901   
       2021-09-30            0.23130            0.17556           0.09635   

                    ...  Earnings Per Share, Basic  \
Ticker Report Date  ...                              
A      2010-07-31   ...                    0.59078   
       2010-10-31   ...                    0.84884   
       2011-01-31   ...                    0.55620   
       2011-04-30   ...                    0.57637   
       2011-07-31   ...                    0.94828   
...                 ...                        ...   
ZYXI   2020-09-30   ...                    0.03514   
       2020-12-31   ...                    0.04669   
       2021-03-31   ...                   -0.01842   
       2021-06-30   ...                    0.07333   
       2021-09-30   ...                    0.15968   

                    Earnings Per Share, Diluted  Sales Per Share  \
Ticker Report Date                                                 
A      2010-07-31                       0.58239          3.98847   
       2010-10-31                       0.82022          4.58140   
       2011-01-31                       0.54366          4.37752   
       2011-04-30                       0.56338          4.83285   
       

In [60]:
# Reset the index
ratios_df.reset_index(level="Ticker",inplace = True)
ratios_df.reset_index(level="Report Date",inplace = True)
# Creating a unique key by combining Ticker and Publish date
ratios_df["unique_key"] = ratios_df["Ticker"]+ratios_df["Publish Date"].astype(str)
# Setting unique_key as Index
ratios_df.set_index("unique_key",inplace = True)
# Show sample data
ratios_df.head()

,Report Date,Ticker,Fiscal Year,Fiscal Period,Publish Date,EBITDA,Total Debt,Free Cash Flow,Gross Profit Margin,Operating Margin,...,"Earnings Per Share, Basic","Earnings Per Share, Diluted",Sales Per Share,Equity Per Share,Free Cash Flow Per Share,Piotroski F-Score,Return On Invested Capital,Cash Return On Invested Capital,Net Debt / EBITDA,Net Debt / EBIT
unique_key,,,,,,,,,,,,,,,,,,,,,
A2010-10-06,2010-07-31,A,2010,Q3,2010-10-06,175000000.0,3678000000,174000000.0,0.52384,0.08309,...,0.59078,0.58239,3.98847,8.07493,0.50144,NaN,0.02668,0.02264,7.77714,11.83478
A2010-12-20,2010-10-31,A,2010,Q4,2010-12-20,270000000.0,3691000000,258000000.0,0.52919,0.12881,...,0.84884,0.82022,4.58140,9.38372,0.75000,6.0,0.03599,0.03180,3.85926,5.13300
A2011-03-09,2011-01-31,A,2011,Q1,2011-03-09,274000000.0,2139000000,65000000.0,0.53720,0.13891,...,0.55620,0.54366,4.37752,9.60231,0.18732,NaN,0.02907,0.00979,-1.82117,-2.36493
A2011-06-07,2011-04-30,A,2011,Q2,2011-06-07,330000000.0,2144000000,393000000.0,0.53667,0.15862,...,0.57637,0.56338,4.83285,11.39193,1.13256,NaN,0.02834,0.05569,-2.51818,-3.12406
A2011-09-07,2011-07-31,A,2011,Q3,2011-09-07,344000000.0,2168000000,74000000.0,0.52750,0.16617,...,0.94828,0.92437,4.85920,12.04598,0.21264,6.0,0.04553,0.01021,-2.71221,-3.32028


### Using simfin API to pull the daily stock price data

In [65]:
# Load daily shareprice data in a DataFrame
prices_df = sf.load_shareprices(variant='daily', market='us')

Dataset "us-shareprices-daily" on disk (3 days old).
- Loading from disk ... Done!


In [66]:
# reset the index to make Ticker a column
prices_df.reset_index(inplace=True)
# Keeping only relevant columns
prices_df = prices_df[["Date","Ticker","Adj. Close"]]
# renaming Adj Close column as Close
prices_df = prices_df.rename(columns={"Adj. Close":"Close"})
# setting Date as index for pct change calculation
prices_df.set_index("Date",inplace = True)
# Calculating the change in price and shifting up by 1
prices_df['pctchange'] = prices_df.groupby('Ticker').Close.pct_change().shift(-1)

df = df[df.line_race != 0]
# Creating a classification price increase/decrease
prices_df['change'] = np.where(prices_df['pctchange'] > 0, 1, 0)
# reset the index to make Date a column
prices_df.reset_index(inplace=True)
# creating unique key by combining Date and Ticker for merging with Ratio date
prices_df["unique_key"] = prices_df["Ticker"]+prices_df["Date"].astype(str)
# setting unique key as index
prices_df.set_index("unique_key",inplace = True)
# Keeping only relevant columns
prices_df = prices_df[["Date","pctchange","change"]]
# dropping NA
prices_df.dropna()
# Show sample data
prices_df.head()

,Date,pctchange,change
unique_key,,,
A2007-01-03,2007-01-03,0.003097,1
A2007-01-04,2007-01-04,-0.009263,0
A2007-01-05,2007-01-05,-0.003562,0
A2007-01-08,2007-01-08,0.001340,1
A2007-01-09,2007-01-09,-0.008925,0


### Creating a DataFrame by merging earning and stock price change data

In [67]:
# merging ratio DataFrame and stock price change DataFrame
all_df = pd.concat([ratios_df,prices_df],axis=1,join="inner")
# drop the rows where there is no change in stock price
all_df = all_df[all_df.pctchange != 0]
# Show sample data
all_df.head()

,Report Date,Ticker,Fiscal Year,Fiscal Period,Publish Date,EBITDA,Total Debt,Free Cash Flow,Gross Profit Margin,Operating Margin,...,Equity Per Share,Free Cash Flow Per Share,Piotroski F-Score,Return On Invested Capital,Cash Return On Invested Capital,Net Debt / EBITDA,Net Debt / EBIT,Date,pctchange,change
unique_key,,,,,,,,,,,,,,,,,,,,,
A2010-10-06,2010-07-31,A,2010,Q3,2010-10-06,175000000.0,3678000000,174000000.0,0.52384,0.08309,...,8.07493,0.50144,NaN,0.02668,0.02264,7.77714,11.83478,2010-10-06,0.016256,1
A2010-12-20,2010-10-31,A,2010,Q4,2010-12-20,270000000.0,3691000000,258000000.0,0.52919,0.12881,...,9.38372,0.75000,6.0,0.03599,0.03180,3.85926,5.13300,2010-12-20,0.004856,1
A2011-03-09,2011-01-31,A,2011,Q1,2011-03-09,274000000.0,2139000000,65000000.0,0.53720,0.13891,...,9.60231,0.18732,NaN,0.02907,0.00979,-1.82117,-2.36493,2011-03-09,-0.025379,0
A2011-06-07,2011-04-30,A,2011,Q2,2011-06-07,330000000.0,2144000000,393000000.0,0.53667,0.15862,...,11.39193,1.13256,NaN,0.02834,0.05569,-2.51818,-3.12406,2011-06-07,0.000955,1
A2011-09-07,2011-07-31,A,2011,Q3,2011-09-07,344000000.0,2168000000,74000000.0,0.52750,0.16617,...,12.04598,0.21264,6.0,0.04553,0.01021,-2.71221,-3.32028,2011-09-07,-0.043367,0


In [ ]:
# Drop columns which doesn't add any value